In [ ]:
from bs4 import BeautifulSoup
import requests 
import time 
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !pip install selenium
# !pip install webdriver-manager
import time
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException


chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


In [ ]:
main_page="https://www.booking.com/searchresults.fr.html?label=gog235jc-1DCAEoggI46AdIDVgDaIwBiAEBmAENuAEXyAEM2AED6AEB-AECiAIBqAIDuAKKst-QBsACAdICJDFjYjRlNmRhLTQ3MDMtNDRiMy05MTNiLWVmZWVkMGZkODRmNdgCBOACAQ&sid=23ea59eaaf9afb886f400ec27b5f1054&aid=397594&sb_lp=1&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Faid%3D397594%3Blabel%3Dgog235jc-1DCAEoggI46AdIDVgDaIwBiAEBmAENuAEXyAEM2AED6AEB-AECiAIBqAIDuAKKst-QBsACAdICJDFjYjRlNmRhLTQ3MDMtNDRiMy05MTNiLWVmZWVkMGZkODRmNdgCBOACAQ%3Bsid%3D23ea59eaaf9afb886f400ec27b5f1054%3Bsb_price_type%3Dtotal%26%3B&ss=Maroc&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=1&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=mar&ac_position=1&ac_langcode=fr&ac_click_type=b&dest_id=143&dest_type=country&place_id_lat=32.4281&place_id_lon=-6.92197&search_pageview_id=43e087aa56b00115&search_selected=true&search_pageview_id=43e087aa56b00115&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&order=class_asc&nflt=class%3D1%3Bclass%3D2%3Bclass%3D3%3Bclass%3D4%3Bclass%3D5"

In [ ]:
# main_page+"&offset=" multiple de 25

In [ ]:
def page_links(start,end,main_url):
    pages=[]
    for i in range(start,end,25):
        if(i==0):
            pages.append(main_url)
        else:
            pages.append(main_page+"&offset="+str(i))
    return pages

In [ ]:
#new version
page_links=page_links(450,600,main_page)
notes=[]
lang=[]
comments=[]
hotels_data=[]
for page in page_links :
    browser= webdriver.Chrome('chromedriver',options=chrome_options)
    browser.get(page)
    try : 
        ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
        WebDriverWait(browser, 10,ignored_exceptions=ignored_exceptions)

    except TimeoutException:
                print("TimeoutException: Element not found")
    urls=[i.get_attribute('href') for i in browser.find_elements(by=By.CLASS_NAME,value='fb01724e5b')]
    hotels=[i.text for i in browser.find_elements(by=By.XPATH,value='.//div[1]/div[2]/div/div/div/div[1]/div/div[1]/div/h3/a/div[1]')]
    for index in range(len(urls)):
        inner_browser= webdriver.Chrome('chromedriver',options=chrome_options)
        inner_browser.get(urls[index])
        try : 
            link = WebDriverWait(inner_browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#guest-featured_reviews__horizontal-block > div > div.hp-featured_reviews-bottom > button')))
            link.click()
            time.sleep(2)
            
            divs=inner_browser.find_elements(by=By.CLASS_NAME , value='review_list_new_item_block')
            for div in divs:
                hotels_data.append(hotels[index])
                comments.append(div.find_element(by=By.CLASS_NAME, value='c-review__inner').text)
                lang.append(div.find_element(by=By.CLASS_NAME, value='c-review__inner').get_attribute('lang')) 
                if(div.find_element(by=By.XPATH, value='.//div[1]/div/div[2]/div/div').text):
                    notes.append(div.find_element(by=By.XPATH, value='.//div/div[2]/div[2]/div[1]/div/div[2]/div/div').text)
                else:
                    notes.append(None)
        except TimeoutException:
            print("TimeoutException: Element not found")
            print("url : ",urls[index] ,"\n ")
            
        
   
    time.sleep(2)



In [ ]:
len(notes)


In [ ]:
page

In [ ]:
data=pd.DataFrame(notes,columns=['notes'])
data.to_csv('notesTemp.csv')
data=pd.DataFrame(comments,columns=['comments'])
data.to_csv('commentsTemp.csv')
data=pd.DataFrame(lang,columns=['lang'])
data.to_csv('langTemp.csv')
data=pd.DataFrame(hotels_data,columns=['hotel_name'])
data.to_csv('hotels_dataTemp.csv')


In [ ]:
data.head()

In [ ]:
# data.to_csv('datakhju8.csv')

In [ ]:
len(hotels_data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   notes       448 non-null    object
 1   comments    448 non-null    object
 2   hotel_name  448 non-null    object
 3   lang        448 non-null    object
dtypes: object(4)
memory usage: 14.1+ KB
